<b>Lab 3 | Week 4 Day 4</b>

Going Deeper:
- Introduce MS Playwright (Browser Automation tool for testing - Next gen Selenium)

---
This is the start of an AWESOME project for Day 5! Really simple and very effective.
- Use Structured output
- Implement a Multi-agent workflow

In [ ]:
from dotenv import load_dotenv
from typing import Annotated, TypedDict
from IPython.display import Image, display

from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition
from langgraph.checkpoint.memory import MemorySaver

from langchain_openai import ChatOpenAI
from langchain.agents import Tool

import requests, os
import gradio as gr

In [ ]:
load_dotenv(override=True)

### Asynchronous LangGraph

To run a tool:  
Sync: `tool.run(inputs)`  
Async: `await tool.arun(inputs)`

To invoke the graph:  
Sync: `graph.invoke(state)`  
Async: `await graph.ainvoke(state)`

In [ ]:
# Define state and create the graph builder

class State(TypedDict):
    messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

In [ ]:
# Create the Push notification tool (same as in lab2)
pushover_token = os.getenv("PUSHOVER_TOKEN")
pushover_user = os.getenv("PUSHOVER_USER")
pushover_url = "https://api.pushover.net/1/messages.json"

def push(text: str):
    """Send a push notification to the user"""
    requests.post(pushover_url, data = {"token": pushover_token, "user": pushover_user, "message": text})

tool_push = Tool(
        name="send_push_notification",
        func=push,
        description="useful when you want to send a push notification"
    )

# tool_push.invoke("Week 4 lab 3 test")

#### Install NodeJs and Playwright

- Playwright needs Node to be installed. Node, npm and npx can be installed in one go by visiting the nodejs.org website and downloading and running the windows installer (.msi package)

node -v     # v22.15.0   (or newer)
npm  -v     # 11.3.0
npx  -v     # 11.3.0

- Playwright is the browser automation software from Microsoft that we use in weeks 4 and 6.

`uv run playwright install`

---
#### And now - after Installing Playwright, a heads up for Windows PC Users:

While executing the next few cells, you might hit a problem with the Playwright browser raising a NotImplementedError.

This should work when we move to python modules, but it can cause problems in Windows in a notebook.

If you it this error and would like to run the notebook, you need to make a small change which seems quite hacky! You need to do this AFTER installing Playwright (prior cells)

1. Right click in `.venv` in the File Explorer on the left and select "Find in folder"
2. Search for `asyncio.set_event_loop_policy(WindowsSelectorEventLoopPolicy())`  
3. That code should be found in a line of code in a file called `kernelapp.py`
4. Comment out the entire else clause that this line is a part of - see the fragment below. Be sure to have the "pass" statement after the ImportError line.
5. Restart the kernel by pressing the "Restart" button above

```python
        if sys.platform.startswith("win") and sys.version_info >= (3, 8):
            import asyncio
 
            try:
                from asyncio import WindowsProactorEventLoopPolicy, WindowsSelectorEventLoopPolicy
            except ImportError:
                pass
                # not affected
           # else:
            #    if type(asyncio.get_event_loop_policy()) is WindowsProactorEventLoopPolicy:
                    # WindowsProactorEventLoopPolicy is not compatible with tornado 6
                    # fallback to the pre-3.8 default of Selector
                    # asyncio.set_event_loop_policy(WindowsSelectorEventLoopPolicy())
```

As an alternative, you can just move to a Python module (which we do anyway in Day 5)

---

#### Introducing nest_asyncio
Python async code only allows for one "event loop" processing aynchronous events.
The `nest_asyncio` library patches this, and is used for special situations, if you need to run a nested event loop.

In [ ]:
import nest_asyncio
nest_asyncio.apply()

### The LangChain community

One of the remarkable things about LangChain is the rich community around it.

Check this out:

In [ ]:
from langchain_community.agent_toolkits import PlayWrightBrowserToolkit
from langchain_community.tools.playwright.utils import create_async_playwright_browser

# If you get a NotImplementedError here or later, see the Heads Up at the top of the notebook

async_browser = create_async_playwright_browser(headless=False) # create browser in headful mode
toolkit = PlayWrightBrowserToolkit.from_browser(async_browser=async_browser)
tools = toolkit.get_tools()

In [ ]:
# Different tools available in the tool kit
for tool in tools:
    print(f"{tool.name}={tool}")

In [ ]:
# Populate the tool info into a dict using dict comprehension

tool_dict = {tool.name: tool for tool in tools}

tool_dict

In [ ]:
# Experiment with a couple of the available tools

navigate_tool = tool_dict.get("navigate_browser")
extract_text_tool = tool_dict.get("extract_text")

await navigate_tool.arun({"url":"https://www.anthropic.com"})
text = await extract_text_tool.arun({})

In [ ]:
import textwrap
print(textwrap.fill(text))

So far, no llms were involved.

Now let's include LLMs in the process

In [ ]:
# Bring the tools together - Playwright tools and the push notification tool
all_tools = tools + [tool_push]

In [ ]:
# Initialize the LLM

llm = ChatOpenAI(model="gpt-4o-mini")
llm_with_tools = llm.bind_tools(all_tools)

In [ ]:
def chatbot(state: State):
    return {"messages": [llm_with_tools.invoke(state["messages"])]}

In [ ]:
graph_builder = StateGraph(State)

graph_builder.add_node('chatbot', chatbot)
graph_builder.add_node('tools', ToolNode(tools=all_tools))

graph_builder.add_conditional_edges('chatbot', tools_condition, 'tools')
graph_builder.add_edge('tools', 'chatbot')
graph_builder.add_edge(START, 'chatbot')

In [ ]:
memory = MemorySaver()
graph = graph_builder.compile(checkpointer=memory)
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:

config = {"configurable": {"thread_id": "10"}}

async def chat(user_input: str, history):
    result = await graph.ainvoke({"messages": [{"role": "user", "content": user_input}]}, config=config)
    return result["messages"][-1].content

In [ ]:
gr.ChatInterface(chat, type="messages").launch()